Run the first 2 code cells only, this would generate a dataset.csv in the current directory and add the label of sonrisa (smile) to all the frames of the video taken, you must re run this process each time you desire to add a new feature to the dataset

In [ ]:
!pip uninstall xgboost

In [4]:
import pandas as pd
import xgboost as xgb
import cv2
import mediapipe as mp

print(pd.__version__,xgb.__version__,cv2.__version__)

1.4.2 1.6.1 4.5.4-dev


In [1]:
!pip install xgboost==1.6.1

ERROR: Could not find a version that satisfies the requirement xgboost==0.9
ERROR: No matching distribution found for xgboost==0.9


In [1]:
import xgboost as xgb
xgb.__version__

'1.6.1'

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb


mp_face_mesh = mp.solutions.face_mesh

class Facemesh:
    def __init__(self, label=None, classifier=None, import_df=True):
        left_eye = [263, 249, 390, 373, 374, 380, 381, 382, 362, 466, 388, 387, 386, 385, 384, 398]
        left_eyebrow = [276, 283, 282, 295, 285, 300, 293, 334, 296, 336]
        lips = [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 185, 40, 39, 37, 0, 267, 269, 270, 409, 78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308, 191, 80, 81, 82, 13, 312, 311, 310, 415]
        right_eye = [33, 7, 163, 144, 145, 153, 154, 155, 133, 246, 161, 160, 159, 158, 157, 173]
        right_eyebrow = [46, 53, 52, 65, 55, 70, 63, 105, 66, 107]
        left_iris = [474, 475, 476, 477]
        right_iris = [469, 470, 471, 472] 
        centers_iris = [468, 473]
        
        self.eyes_gestures_kps = left_eye + left_eyebrow + right_eye + right_eyebrow
        self.kps = self.eyes_gestures_kps + lips
        self.eyes_kps = self.eyes_gestures_kps + left_iris + right_iris
        self.mouth = lips
        mp_face_mesh = mp.solutions.face_mesh
        self.label = label
        self.import_df = import_df
        self.face_mesh = mp_face_mesh.FaceMesh(  max_num_faces=1,
                                            refine_landmarks=True,
                                            min_detection_confidence=0.5,
                                            min_tracking_confidence=0.5)
        self.class_names = ['boca_abierta','neutral', 'ojo_derecho', 'ojo_izquierdo', 'sonrisa']
        classifiers = {
            'eye_gesture_prediction': {
                'class_names' : ['neutral','ojo_derecho','ojo_izquierdo'],
                'file_name' : 'modelos/eye_gesture_model_3.json',
                'columns': [2, 3, 10, 11, 18, 19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 94, 95, 96, 97, 98, 99, 106, 107, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 121, 122, 123, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 182, 183],
                'df_name': 'eyes_gestures_ds.csv',
                'kps' : self.eyes_gestures_kps
            },
            'kps_original': {
                'class_names':['boca_abierta','neutral', 'ojo_derecho', 'ojo_izquierdo', 'sonrisa'],
                'file_name' : 'modelos/model_2.json',
                'columns' : list(range(184)),
                'kps' : self.kps
            },
            'lips_gesture_prediction': {
                'class_names': ['boca_abierta', 'neutral', 'sonrisa'],
                'file_name' : 'modelos/lips_gesture_model_1.json',
                'columns': [0, 1, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 26, 27, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 64, 65, 86, 87, 88, 89, 90, 91, 92, 93, 100, 101, 102, 103, 104, 105, 114, 115, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 152, 153, 174, 175, 176, 177, 178, 179, 180, 181],
                'df_name': 'lips_ds.csv',
                'kps' : self.mouth
            }
        }
        if classifier:
            self.classifier = classifiers[classifier]
            self.load_xgboost_model() 
        
        
    def load_xgboost_model(self):
        file_name = self.classifier['file_name']
        model = xgb.Booster()
        model.load_model(file_name)
        self.xgb_model = model
        
    def get_image_landmarks(self,image):
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.face_mesh.process(image)  ##Magic happening
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        return results
            
    def insert_dataframe(self, points):
        df_name = self.classifier['df_name']
        df = pd.DataFrame(points).transpose()
        df.columns = self.classifier['columns']+[184]
        df.to_csv(df_name,mode='a',header=not self.import_df, index = False) 
        
                        
    def draw_image_kps(self,image,facial_landmarks):
        height, width, _ = image.shape
        points = []
        for i in range(len(facial_landmarks.landmark)):
            if i in self.classifier['kps']:
                pt= facial_landmarks.landmark[i]
                points.append(pt.x)
                points.append(pt.y)
                x = int(pt.x * width)
                y = int(pt.y * height)
                cv2.circle(image, (x, y), 1, (0, 100, 255), -1)
        return points
    
    def show_image(self, image):
        cv2.imshow("Image", image)
        cv2.waitKey(1)
    
    def draw_and_insert_kps(self,image,facial_landmarks):
        points = self.draw_image_kps(image,facial_landmarks)
        points.append(self.label)
        cv2.flip(image, 1)
        self.show_image(image)
        self.insert_dataframe(points)
    
    def draw_and_predict_gesture(self,image,facial_landmarks):
        points = self.draw_image_kps(image,facial_landmarks)
        df = self.assemble_df(points)
        prediction = self.predict_gesture(df)
        cv2.flip(image, 1)
        cv2.putText(image,prediction,(200,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))
        self.show_image(image)
        
    def predict_gesture(self,df):
        formatted_df = xgb.DMatrix(df)
        prediction = int(self.xgb_model.predict(formatted_df))
        return self.classifier['class_names'][prediction]
    
    def assemble_df(self, points):
        df = pd.DataFrame(points).transpose()
        df.columns = self.classifier['columns']
        return df


In [2]:
import pandas as pd
import xgboost as xgb
import mediapipe as mp


mp_face_mesh = mp.solutions.face_mesh

class Facemesh_V2(Facemesh):
    def __init__(self):
        super().__init__()
        self.make_kps_dict()
    
    def make_kps_dict(self):
        self.whole_face = {
            'eye_gesture_prediction': {
                'class_names' : ['neutral','ojo_derecho','ojo_izquierdo'],
                'file_name' : '../modelos/eye_gesture_model_3.json',
                'columns': [2, 3, 10, 11, 18, 19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 94, 95, 96, 97, 98, 99, 106, 107, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 121, 122, 123, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 182, 183],
                'df_name': 'eyes_gestures_ds.csv',
                'kps' : self.eyes_gestures_kps
            },
            'lips_gesture_prediction': {
                'class_names': ['boca_abierta', 'neutral', 'sonrisa'],
                'file_name' : '../modelos/lips_gesture_model_1.json',
                'columns': [0, 1, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 26, 27, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 64, 65, 86, 87, 88, 89, 90, 91, 92, 93, 100, 101, 102, 103, 104, 105, 114, 115, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 152, 153, 174, 175, 176, 177, 178, 179, 180, 181],
                'df_name': 'lips_ds.csv',
                'kps' : self.mouth
            }
        }
        self.whole_face['kps'] = self.kps
        self.load_models('eye_gesture_prediction')
        self.load_models('lips_gesture_prediction')
    
    def load_models(self,classifier_name):
        file_name = self.whole_face[classifier_name]['file_name']
        model = xgb.Booster()
        model.load_model(file_name)
        self.whole_face[classifier_name]['xgb'] = model
    
    def print_and_predict_gestures(self,image,facial_landmarks):
        points = self.get_points(image.shape,facial_landmarks)
        df_eyes, df_mouth = self.assemble_dfs(points)
        eye_prediction = self.predict_gesture(df_eyes,'eye_gesture_prediction')
        mouth_prediction = self.predict_gesture(df_mouth,'lips_gesture_prediction')
        print("EYE PREDICTION",eye_prediction,"MOUTH PREDICTION",mouth_prediction)
    
    def assemble_dfs(self,points):
        dfs = pd.DataFrame(points).transpose()
        df_eyes = dfs[self.whole_face['eye_gesture_prediction']['columns']]
        df_mouth = dfs[self.whole_face['lips_gesture_prediction']['columns']]
        return df_eyes, df_mouth 
    
    def get_points(self,shape,facial_landmarks):
        height, width, _ = shape
        points = []
        for i in range(len(facial_landmarks.landmark)):
            if i in self.whole_face['kps']:
                pt= facial_landmarks.landmark[i]
                points.append(pt.x)
                points.append(pt.y)
                x = int(pt.x * width)
                y = int(pt.y * height)
        return points
        
    def predict_gesture(self,df,classifier_name):
        formatted_df = xgb.DMatrix(df)
        prediction = int(self.whole_face[classifier_name]['xgb'].predict(formatted_df))
        return self.whole_face[classifier_name]['class_names'][prediction]


In [4]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
#lips_gesture_prediction, kps_original, eye_gesture_prediction

face_mesh = Facemesh_V2()
num_frames = 0
max_frames = 500
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    
    if num_frames>=max_frames:
        break
    else:
        num_frames+=1
   
    multi_face_lm = face_mesh.get_image_landmarks(image)
    if multi_face_lm.multi_face_landmarks:
        for face_landmarks in multi_face_lm.multi_face_landmarks:
            face_mesh.print_and_predict_gestures(image,face_landmarks) 
cv2.destroyAllWindows()
cap.release()

EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION neutral
EYE PREDICTION neutral MOUTH PREDICTION 

EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION neutral MOUTH PREDICTION boca_abierta
EYE PREDICTION ojo_derecho MOUTH PREDICTION bo

EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo MOUTH PREDICTION sonrisa
EYE PREDICTION ojo_izquierdo

In [4]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)
#lips_gesture_prediction, kps_original, eye_gesture_prediction

face_mesh = Facemesh_V2("ojo_izquierdo",'eye_gesture_prediction',import_df = True)
num_frames = 0
max_frames = 200
do_classification = True
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    
    if num_frames>=max_frames:
        break
    else:
        num_frames+=1
   
    multi_face_lm = face_mesh.get_image_landmarks(image)
    if multi_face_lm.multi_face_landmarks:
        for face_landmarks in multi_face_lm.multi_face_landmarks:
            if do_classification:
                face_mesh.draw_and_predict_gesture(image,face_landmarks)
            else:
                face_mesh.draw_and_insert_kps(image,face_landmarks)
cv2.destroyAllWindows()
cap.release()

In [8]:
##Separate dataset into individual datasets
fm = Facemesh("ojo_izquierdo",import_df = True)

df = pd.read_csv('dataset_01.csv')


def get_index_of_array_in_order(scrambled_list,content_list):
    scrambled_list.sort()
    #content_list.sort() #Si se desea ver la lista resultante en orden
    return [scrambled_list.index(element) for element in content_list]

def adjust_for_two_values(array):
    new_list = []
    for index in array:
        value = index*2
        new_list.append(value)
        new_list.append(value+1)
    return new_list

def array_content_to_str(array):
    return [str(index) for index in array]
    
def sort_df_by_col_list(df,col_list):
    columns = df.columns[df.columns.isin(col_list+['184'])] #Add 185 bc thats the class
    return df[columns]
#print(get_index_of_array_in_order(fm.kps,fm.mouth))

#print(lips_df)

lips_array = array_content_to_str( adjust_for_two_values( get_index_of_array_in_order(fm.kps,fm.mouth) ) )
lips_df = sort_df_by_col_list(df,lips_array)
lips_df = lips_df[lips_df['184'] != 'ojo_izquierdo' ]
lips_df = lips_df[lips_df['184'] != 'ojo_derecho' ]
print(lips_df)
a = adjust_for_two_values( get_index_of_array_in_order(fm.kps,fm.eyes_gestures_kps) )
a.sort()
print(a)
a = adjust_for_two_values( get_index_of_array_in_order(fm.kps,fm.mouth) )
a.sort()
print(a)
eyes_array = array_content_to_str( adjust_for_two_values( get_index_of_array_in_order(fm.kps,fm.eyes_gestures_kps) ) )
eyes_df = sort_df_by_col_list(df,eyes_array)

eyes_df = eyes_df[eyes_df['184'] != 'boca_abierta' ]
eyes_df = eyes_df[eyes_df['184'] != 'sonrisa' ]
print(eyes_df)

#eyes_df.to_csv('eyes_gestures_ds.csv',mode='a',index = False)
#lips_df.to_csv('lips_ds.csv',mode='a',index = False)

             0         1         4         5         6         7         8  \
0     0.544275  0.489766  0.544920  0.507024  0.544717  0.507232  0.545840   
1     0.534347  0.493370  0.535169  0.508110  0.535000  0.508246  0.535752   
2     0.533318  0.494122  0.534231  0.509847  0.533963  0.510075  0.534930   
3     0.532908  0.495667  0.533887  0.510025  0.533832  0.510170  0.535072   
4     0.532545  0.494852  0.533831  0.509942  0.533748  0.510049  0.535094   
...        ...       ...       ...       ...       ...       ...       ...   
4495  0.543846  0.561155  0.543343  0.579029  0.543975  0.646547  0.543592   
4496  0.543188  0.559785  0.542870  0.578905  0.543899  0.646822  0.543669   
4497  0.542702  0.559950  0.542621  0.578734  0.544262  0.645952  0.544289   
4498  0.543265  0.560728  0.542977  0.578296  0.544505  0.647686  0.544373   
4499  0.542427  0.560136  0.542208  0.578968  0.543892  0.646901  0.543650   

             9        12        13  ...       153       174    

In [27]:
fm.kps[32]

91

In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

face_mesh = Facemesh("ojo_izquierdo",import_df = True)
num_frames = 0
max_frames = 500

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue
    
    if num_frames>=max_frames:
        break
    else:
        num_frames+=1
   
    multi_face_lm = face_mesh.get_image_landmarks(image)
    if multi_face_lm.multi_face_landmarks:
        for face_landmarks in multi_face_lm.multi_face_landmarks:
            face_mesh.draw_image_kps(image,face_landmarks)
cv2.destroyAllWindows()
cap.release()

This is for getting all the points that we desire as the contour

In [3]:
import pandas as pd
df = pd.read_csv('dataset.csv')
df

,Unnamed: 0,0,7,13,14,17,33,37,39,40,...,387,388,390,398,402,405,409,415,466,label
0,0.0,0.524771,0.472195,0.524074,0.524118,0.524266,0.469790,0.514229,0.504052,0.496528,...,0.563230,0.566700,0.564955,0.541406,0.539144,0.541877,0.552934,0.548931,0.568775,sonrisa
1,1.0,0.677439,0.586151,0.684651,0.684459,0.694251,0.584403,0.675642,0.677446,0.678936,...,0.574215,0.576326,0.582930,0.578896,0.682753,0.690420,0.678271,0.680739,0.577953,sonrisa
2,NaN,0.000000,7.000000,13.000000,14.000000,17.000000,33.000000,37.000000,39.000000,40.000000,...,387.000000,388.000000,390.000000,398.000000,402.000000,405.000000,409.000000,415.000000,466.000000,label
3,0.0,0.522667,0.472494,0.522497,0.522482,0.522313,0.470072,0.512364,0.502720,0.496089,...,0.564420,0.567975,0.566535,0.542418,0.538375,0.541396,0.553187,0.548987,0.570100,sonrisa
4,1.0,0.676059,0.586140,0.685241,0.684971,0.697718,0.584078,0.673952,0.675535,0.677193,...,0.573736,0.575647,0.582090,0.578318,0.683179,0.693237,0.677420,0.680575,0.577130,sonrisa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,0.0,0.489493,0.418754,0.489987,0.490109,0.490022,0.415088,0.477244,0.465332,0.457396,...,0.570004,0.575526,0.573304,0.531672,0.510412,0.515203,0.531063,0.524800,0.578488,neutro
1774,1.0,0.592383,0.431372,0.610819,0.611286,0.635471,0.426097,0.587606,0.593914,0.601461,...,0.419230,0.423404,0.436741,0.429842,0.612255,0.631938,0.613018,0.616248,0.426623,neutro
1775,NaN,0.000000,7.000000,13.000000,14.000000,17.000000,33.000000,37.000000,39.000000,40.000000,...,387.000000,388.000000,390.000000,398.000000,402.000000,405.000000,409.000000,415.000000,466.000000,label
1776,0.0,0.490287,0.419085,0.490634,0.490685,0.490486,0.415381,0.478176,0.466257,0.458290,...,0.570479,0.575912,0.573586,0.533126,0.510745,0.515446,0.531121,0.524867,0.578864,neutro


In [21]:

left_eye = [(263, 249), (249, 390), (390, 373), (373, 374),
                               (374, 380), (380, 381), (381, 382), (382, 362),
                               (263, 466), (466, 388), (388, 387), (387, 386),
                               (386, 385), (385, 384), (384, 398), (398, 362)]

FACEMESH_LIPS = [(61, 146), (146, 91), (91, 181), (181, 84), (84, 17),
                           (17, 314), (314, 405), (405, 321), (321, 375),
                           (375, 291), (61, 185), (185, 40), (40, 39), (39, 37),
                           (37, 0), (0, 267),
                           (267, 269), (269, 270), (270, 409), (409, 291),
                           (78, 95), (95, 88), (88, 178), (178, 87), (87, 14),
                           (14, 317), (317, 402), (402, 318), (318, 324),
                           (324, 308), (78, 191), (191, 80), (80, 81), (81, 82),
                           (82, 13), (13, 312), (312, 311), (311, 310),
                           (310, 415), (415, 308)]
LEFT_EYEBROW = [(276, 283), (283, 282), (282, 295),
                                   (295, 285), (300, 293), (293, 334),
                                   (334, 296), (296, 336)]
RIGHT_EYE = [(33, 7), (7, 163), (163, 144), (144, 145),
                                (145, 153), (153, 154), (154, 155), (155, 133),
                                (33, 246), (246, 161), (161, 160), (160, 159),
                                (159, 158), (158, 157), (157, 173), (173, 133)]
RIGHT_EYEBROW = [(46, 53), (53, 52), (52, 65), (65, 55),
                                    (70, 63), (63, 105), (105, 66), (66, 107)]


FACEMESH_LEFT_IRIS = [(474, 475), (475, 476), (476, 477), (477, 474)]
FACEMESH_RIGHT_IRIS = [(469, 470), (470, 471), (471, 472), (472, 469)]

a = []
for i in FACEMESH_RIGHT_IRIS:
    if i[0] not in a:
        a.append(i[0])
    if  i[1] not in a:
        a.append(i[1])
print(a)

[469, 470, 471, 472]


This fragments of code should not be runned just examples and past code

In [ ]:
import cv2
import mediapipe as mp

cap = cv2.VideoCapture(0)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()
 
center_ids = [468, 473]

while True:
    # Image
    
    ret, image = cap.read()
    if ret is not True:
        break
    
    height, width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_image)
    
    if result.multi_face_landmarks != None:

        for facial_landmarks in result.multi_face_landmarks:
            for i in range(len(facial_landmarks.landmark)):
                pt1 = facial_landmarks.landmark[i]
                x = int(pt1.x * width)
                y = int(pt1.y * height)
                cv2.circle(image, (x, y), 1, (100, 100, 0), -1)
    
    cv2.imshow("Image", image)
    cv2.waitKey(1)

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

left_eye = [263, 249, 390, 373, 374, 380, 381, 382, 466, 388, 387, 386, 385, 384, 398]

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
